In [1]:
import re
import logging
from numpy import nan
from geopy.geocoders import GoogleV3
from geopy.exc import GeocoderTimedOut

In [ ]:
# Configure logging settings
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

file_handler = logging.FileHandler('geolocate_status.log')
formatter = logging.Formatter('%(asctime)s::%(levelname)s::%(name)s::%(message)s')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

In [ ]:
with open('google-maps-api/api-key.txt', 'r') as f:
    key = f.readline()

# Setup Geolocator
geolocator = GoogleV3(api_key=key, user_agent="uchicago_surronding_area", format_string="%s, Chicago, IL")

In [ ]:
# Adds "avenue" to addresss
# It helps narrow down location

avenues = 'cottage grove,drexel,ingleside,ellis,greenwood,university,\
                woodlawn,kimbark,kenwood,dorchester,blackstone,harper,\
                lake park,stony island,cornell,everett'.split(',')

def addAvenue(address):
    
    for ave in avenues:
        if re.search(ave, address):
            address += ' avenue'
            break

    return address

In [ ]:
def address2coordinates(address):
    '''
    Function takes an address as a string and formats to improve 
    likelihood of getting gps coordinatesd.

    Returns
    -------
    If location found: a tuple containing the latitude & longitude
    If not: an integer code (user defined)
    '''
    
    address = re.sub(r' \(.*\)', '', address.lower())  # remove parenthetical info
    address = re.sub(r' at|and ', ' & ', address)      # fix intersection
    
    try:
        # Check if address is between streets
        # E.g. (52nd St. between Greenwood & University)
        pattern = re.findall(r'(\w.*(?= between)) between ((?<=between )\w.*(?= &)) & ((?<=\& )\w.*)', address)[0]
        street_1 = f"{pattern[0]} {pattern[1]}"
        street_2 = f"{pattern[0]} {pattern[2]}"
        
        # Server timeout time is 10sec
        location_1 = geolocator.geocode(addAvenue(street_1), timeout=10)
        location_2 = geolocator.geocode(addAvenue(street_2), timeout=10)
        
        # Returns midpoint coordinates between the two locations
        return ((location_1.latitude + location_2.latitude)/2, 
                (location_1.longitude + location_2.longitude)/2)
            
    except IndexError:
        # Only single address
        location = geolocator.geocode(address, timeout=10)
        if location != None:
            return (location.latitude, location.longitude)
        
    except GeocoderTimedOut as timeout:
        # Server timeout value if any (check later)
        logger.info(f"{timeout}::{address}")
        return nan
    
    except Exception as err:
        # error value will point to unforseen error if any (check later)
        logger.info(f"")
        print('An error occurred!')
        return 999
    
    else:
        return nan